<div align="center"><h1>Accelerating Applications with CUDA C/C++</h1></div>

---
## Objectives

By the time you complete this lab, you will be able to:

- Write, compile, and run C/C++ programs that both call CPU functions and **launch** GPU **kernels**.
- Control parallel **thread hierarchy** using **execution configuration**.
- Refactor serial loops to execute their iterations in parallel on a GPU.
- Allocate and free memory available to both CPUs and GPUs.
- Handle errors generated by CUDA code.
- Accelerate CPU-only applications.

---
## Accelerated Systems

*Accelerated systems*, also referred to as *heterogeneous systems*, are those composed of both CPUs and GPUs. Accelerated systems run CPU programs which in turn, launch functions that will benefit from the massive parallelism provided by GPUs. This lab environment is an accelerated system which includes an NVIDIA GPU. Information about this GPU can be queried with the `nvidia-smi` (*Systems Management Interface*) command line command.

In [1]:
!nvidia-smi

Wed Jul 26 10:40:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID T4-8C          On   | 00000000:00:09.0 Off |                    0 |
| N/A   N/A    P0    N/A /  N/A |   1104MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

---
## Writing Application Code for the GPU

CUDA provides extensions for many common programming languages, in the case of this lab, C/C++. These language extensions easily allow developers to run functions in their source code on a GPU.

Below is a `.cu` file (`.cu` is the file extension for CUDA-accelerated programs). It contains two functions, the first which will run on the CPU, the second which will run on the GPU.

```cpp
void CPUFunction()
{
  printf("This function is defined to run on the CPU.\n");
}

__global__ void GPUFunction()
{
  printf("This function is defined to run on the GPU.\n");
}

int main()
{
  CPUFunction();

  GPUFunction<<<1, 1>>>();
  cudaDeviceSynchronize();
}
```

`__global__ void GPUFunction()`
  - The `__global__` keyword indicates that the following function will run on the GPU, and can be invoked **globally**, which in this context means either by the CPU, or, by the GPU.
  - Often, code executed on the CPU is referred to as **host** code, and code running on the GPU is referred to as **device** code.
  - It is required that functions defined with the `__global__` keyword return type `void`.

`GPUFunction<<<1, 1>>>();`
  - Typically, when calling a function to run on the GPU, we call this function a **kernel**, which is **launched**.
  - When launching a kernel, we must provide an **execution configuration**, which is done by using the `<<< ... >>>` syntax just prior to passing the kernel any expected arguments.
  - At a high level, execution configuration allows programmers to specify the **thread hierarchy** for a kernel launch, which defines the number of thread groupings (called **blocks**), as well as how many **threads** to execute in each block. The kernel is launching with `1` block of threads (the first execution configuration argument) which contains `1` thread (the second configuration argument).

`cudaDeviceSynchronize();`
  - Unlike much C/C++ code, launching kernels is **asynchronous**: the CPU code will continue to execute *without waiting for the kernel launch to complete*.
  - A call to `cudaDeviceSynchronize`, a function provided by the CUDA runtime, will cause the host (CPU) code to wait until the device (GPU) code completes, and only then resume execution on the CPU.

---
### Exercise: Write a Hello GPU Kernel

The [`01-hello-gpu.cu`](../edit/01-hello/01-hello-gpu.cu) contains a program. The comments in [`01-hello-gpu.cu`](../edit/01-hello/01-hello-gpu.cu) will assist your work. If you get stuck, or want to check your work, refer to the [solution](../edit/01-hello/solutions/01-hello-gpu-solution.cu).

In [2]:
!nvcc -arch=sm_70 -o hello-gpu ../edit/01-hello/01-hello-gpu.cu -run

gcc: error: ./01-hello/01-hello-gpu.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.


After successfully refactoring [`01-hello-gpu.cu`](../edit/01-hello/01-hello-gpu.cu), make the following modifications, attempting to compile and run it after each change. When given errors, take the time to read them carefully.

- Remove the keyword `__global__` from your kernel definition.
- Remove the execution configuration.
- Remove the call to `cudaDeviceSynchronize`.
- Refactor `01-hello-gpu.cu` so that `Hello from the GPU` prints **before** `Hello from the CPU`.
- Refactor `01-hello-gpu.cu` so that `Hello from the GPU` prints **twice**, once  **before** `Hello from the CPU`, and once **after**.

---
### Compiling and Running Accelerated CUDA Code

This section contains details about the `nvcc` command you issued above to compile and run your `.cu` program.

The CUDA platform ships with the [**NVIDIA CUDA Compiler**](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html) `nvcc`, which can compile CUDA accelerated applications, both the host, and the device code they contain.

`nvcc` will be very familiar to experienced `gcc` users. Compiling, for example, a `some-CUDA.cu` file, is simply:

`nvcc -arch=sm_70 -o out some-CUDA.cu -run`
  - `nvcc` is the command line command for using the `nvcc` compiler.
  - `some-CUDA.cu` is passed as the file to compile.
  - The `o` flag is used to specify the output file for the compiled program.
  - The `arch` flag indicates for which **architecture** the files must be compiled. For the present case `sm_70` will serve to compile specifically for the GPU this lab is running on, but for those interested in a deeper dive, please refer to the docs about the [`arch` flag](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#options-for-steering-gpu-code-generation), [virtual architecture features](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature-list) and [GPU features](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature-list).
  - As a matter of convenience, providing the `run` flag will execute the successfully compiled binary.

---
## Launching Parallel Kernels

The execution configuration allows programmers to specify how many groups of threads - called **thread blocks**, or just **blocks** - and how many threads they would like each thread block to contain. The syntax for this is:

`<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS_PER_BLOCK>>>`

** The kernel code is executed by every thread in every thread block configured when the kernel is launched**.

---
### Exercise: Launch Parallel Kernels

[`01-first-parallel.cu`](../edit/02-first-parallel/01-first-parallel.cu) currently makes a very basic function call that prints the message `This should be running in parallel.` Refer to [the solution](../edit/02-first-parallel/solutions/01-first-parallel-solution.cu) if you get stuck.

- Refactor the `firstParallel` function to launch as a CUDA kernel on the GPU. You should still be able to see the output of the function after compiling and running `01-first-parallel.cu` with the `nvcc` command just below.
- Refactor the `firstParallel` kernel to execute in parallel on 5 threads, all executing in a single thread block. You should see the output message printed 5 times after compiling and running the code.
- Refactor the `firstParallel` kernel again, this time to execute in parallel inside 5 thread blocks, each containing 5 threads. You should see the output message printed 25 times now after compiling and running.

In [5]:
!nvcc -arch=sm_70 -o first-parallel ../edit/02-first-parallel/01-first-parallel.cu -run

This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.
This should be running in parallel.


---
## Thread and Block Indices

Each thread is given an index within its thread block, starting at `0`. Additionally, each block is given an index, starting at `0`. Just as threads are grouped into thread blocks, blocks are grouped into a **grid**, which is the highest entity in the CUDA thread hierarchy. In summary, CUDA kernels are executed in a grid of 1 or more blocks, with each block containing the same number of 1 or more threads.

CUDA kernels have access to special variables identifying both the index of the thread (within the block) that is executing the kernel, and, the index of the block (within the grid) that the thread is within. These variables are `threadIdx.x` and `blockIdx.x` respectively.

---
### Exercise: Use Specific Thread and Block Indices

Currently the [`01-thread-and-block-idx.cu`](../edit/03-indices/01-thread-and-block-idx.cu) file contains a working kernel that is printing a failure message. After refactoring, compile and run the code with the code execution cell below to confirm your work. Refer to [the solution](../edit/03-indices/solutions/01-thread-and-block-idx-solution.cu) if you get stuck.

In [15]:
!nvcc -arch=sm_70 -o thread-and-block-idx ../edit/03-indices/01-thread-and-block-idx.cu -run

Success!


---
## Accelerating For Loops

Consider the following for loop:

```cpp
int N = 2<<20;
for (int i = 0; i < N; ++i)
{
  printf("%d\n", i);
}
```

In order to parallelize this loop, 2 steps must be taken:

- A kernel must be written to do the work of a **single iteration of the loop**.
- Because the kernel will be agnostic of other running kernels, the execution configuration must be such that the kernel executes the correct number of times, for example, the number of times the loop would have iterated.

---
### Exercise: Accelerating a For Loop with a Single Block of Threads

Currently, the `loop` function inside [`01-single-block-loop.cu`](../edit/04-loops/01-single-block-loop.cu), runs a for loop that will serially print the numbers `0` through `9`. After successfully refactoring, the numbers `0` through `9` should still be printed. Refer to [the solution](../edit/04-loops/solutions/01-single-block-loop-solution.cu) if you get stuck.

In [12]:
!nvcc -arch=sm_70 -o single-block-loop ../edit/04-loops/01-single-block-loop.cu -run

This is iteration number 0
This is iteration number 1
This is iteration number 2
This is iteration number 3
This is iteration number 4
This is iteration number 5
This is iteration number 6
This is iteration number 7
This is iteration number 8
This is iteration number 9


---
## Using Block Dimensions for More Parallelization

There is a limit to the number of threads that can exist in a thread block: 1024 to be precise.

CUDA Kernels have access to a special variable that gives the number of threads in a block: `blockDim.x`. Using this variable, in conjunction with `blockIdx.x` and `threadIdx.x`, increased parallelization can be accomplished by organizing parallel execution across multiple blocks of multiple threads with the idiomatic expression `threadIdx.x + blockIdx.x * blockDim.x`. 

---
### Exercise: Accelerating a For Loop with Multiple Blocks of Threads

Currently, the `loop` function inside [`02-multi-block-loop.cu`](../edit/04-loops/02-multi-block-loop.cu) runs a for loop that will serially print the numbers `0` through `9`. After successfully refactoring, the numbers `0` through `9` should still be printed. For this exercise, as an additional constraint, use an execution configuration that launches *at least 2 blocks of threads.* Refer to [the solution](../edit/04-loops/solutions/02-multi-block-loop-solution.cu) if you get stuck.

In [17]:
!nvcc -arch=sm_70 -o multi-block-loop ../edit/04-loops/02-multi-block-loop.cu -run

This is iteration number 5
This is iteration number 6
This is iteration number 7
This is iteration number 8
This is iteration number 9
This is iteration number 0
This is iteration number 1
This is iteration number 2
This is iteration number 3
This is iteration number 4


---
## Allocating Memory to be accessed on the GPU and the CPU

More recent versions of CUDA (version 6 and later) have made it easy to allocate memory that is available to both the CPU host and any number of GPU devices, and while there are many [intermediate and advanced techniques](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations) for memory management that will support the most optimal performance in accelerated applications, the most basic CUDA memory management technique we will now cover supports fantastic performance gains over CPU-only applications with almost no developer overhead.

To allocate and free memory, and obtain a pointer that can be referenced in both host and device code, replace calls to `malloc` and `free` with `cudaMallocManaged` and `cudaFree` as in the following example:

```cpp
// CPU-only

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
a = (int *)malloc(size);

// Use `a` in CPU-only program.

free(a);
```

```cpp
// Accelerated

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
// Note the address of `a` is passed as first argument.
cudaMallocManaged(&a, size);

// Use `a` on the CPU and/or on any GPU in the accelerated system.

cudaFree(a);
```

---
### Exercise: Array Manipulation on both the Host and Device

The [`01-double-elements.cu`](../edit/05-allocate/01-double-elements.cu) program allocates an array, initializes it with integer values on the host, attempts to double each of these values in parallel on the GPU, and then confirms whether or not the doubling operations were successful, on the host. Currently the program will not work: it is attempting to interact on both the host and the device with an array at pointer `a`, but has only allocated the array (using `malloc`) to be accessible on the host. Refactor the application to meet the following conditions, referring to [the solution](../edit/05-allocate/solutions/01-double-elements-solution.cu) if you get stuck:

- `a` should be available to both host and device code.
- The memory at `a` should be correctly freed.

In [25]:
!nvcc -arch=sm_70 -o double-elements ../edit/05-allocate/01-double-elements.cu -run

nvcc error   : './"double-elements"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"double-elements"' core dumped


---
## Handling Block Configuration Mismatches to Number of Needed Threads

It may be the case that an execution configuration cannot be expressed that will create the exact number of threads needed for parallelizing a loop.

A common example has to do with the desire to choose optimal block sizes. For example, due to GPU hardware traits, blocks that contain a number of threads that are a multiple of 32 are often desirable for performance benefits.

Here is an example of an idiomatic way to write an execution configuration when both `N` and the number of threads in a block are known, and an exact match between the number of threads in the grid and `N` cannot be guaranteed. It ensures that there are always at least as many threads as needed for `N`, and only 1 additional block's worth of threads extra, at most:

```cpp
// Assume `N` is known
int N = 100000;

// Assume we have a desire to set `threads_per_block` exactly to `256`
size_t threads_per_block = 256;

// Ensure there are at least `N` threads in the grid, but only 1 block's worth extra
size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

some_kernel<<<number_of_blocks, threads_per_block>>>(N);
```

Because the execution configuration above results in more threads in the grid than `N`, care will need to be taken inside of the `some_kernel` definition so that `some_kernel` does not attempt to access out of range data elements, when being executed by one of the "extra" threads:

```cpp
__global__ some_kernel(int N)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;

  if (idx < N) // Check to make sure `idx` maps to some value within `N`
  {
    // Only do work if it does
  }
}
```

---
### Exercise: Accelerating a For Loop with a Mismatched Execution Configuration

The program in [`02-mismatched-config-loop.cu`](../edit/05-allocate/02-mismatched-config-loop.cu) allocates memory, using `cudaMallocManaged` for a 1000 element array of integers, and then seeks to initialize all the values of the array in parallel using a CUDA kernel. This program assumes that both `N` and the number of `threads_per_block` are known. Your task is to complete the following two objectives, refer to [the solution](../edit/05-allocate/solutions/02-mismatched-config-loop-solution.cu) if you get stuck:

- Assign a value to `number_of_blocks` that will make sure there are at least as many threads as there are elements in `a` to work on.
- Update the `initializeElementsTo` kernel to make sure that it does not attempt to work on data elements that are out of range.

In [1]:
!nvcc -arch=sm_70 -o mismatched-config-loop ../edit/05-allocate/02-mismatched-config-loop.cu -run

nvcc error   : './"mismatched-config-loop"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"mismatched-config-loop"' core dumped


---
## Data Sets Larger Than the Grid

Either by choice, often to create the most performant execution configuration, or out of necessity, the number of threads in a grid may be smaller than the size of a data set. Consider an array with 1000 elements, and a grid with 250 threads. Here, each thread in the grid will need to be used 4 times. One common method to do this is to use a **grid-stride loop** within the kernel.

In a grid-stride loop, each thread will calculate its unique index within the grid using `tid+bid*bdim`, perform its operation on the element at that index within the array, and then, add to its index the number of threads in the grid and repeat, until it is out of range of the array.

CUDA provides a special variable giving the number of blocks in a grid, `gridDim.x`. Calculating the total number of threads: `gridDim.x * blockDim.x`. With this in mind, here is a verbose example of a grid-stride loop within a kernel:

```cpp
__global__ void kernel(int *a, int N)
{
  int indexWithinTheGrid = threadIdx.x + blockIdx.x * blockDim.x;
  int gridStride = gridDim.x * blockDim.x;

  for (int i = indexWithinTheGrid; i < N; i += gridStride)
  {
    // do work on a[i];
  }
}
```

---
### Exercise: Use a Grid-Stride Loop to Manipulate an Array Larger than the Grid

Refactor [`03-grid-stride-double.cu`](../edit/05-allocate/03-grid-stride-double.cu) to use a grid-stride loop in the `doubleElements` kernel. Refer to [the solution](../edit/05-allocate/solutions/03-grid-stride-double-solution.cu) if you get stuck.

In [2]:
!nvcc -arch=sm_70 -o grid-stride-double ../edit/05-allocate/03-grid-stride-double.cu -run

nvcc error   : './"grid-stride-double"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"grid-stride-double"' core dumped


---
## Error Handling

Many, if not most CUDA functions (see, for example, the [memory management functions](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY)) return a value of type `cudaError_t`. Here is an example where error handling is performed for a call to `cudaMallocManaged`:

```cpp
cudaError_t err;
err = cudaMallocManaged(&a, N)                    // Assume the existence of `a` and `N`.

if (err != cudaSuccess)                           // `cudaSuccess` is provided by CUDA.
{
  printf("Error: %s\n", cudaGetErrorString(err)); // `cudaGetErrorString` is provided by CUDA.
}
```

Launching kernels, which are defined to return `void`, do not return a value of type `cudaError_t`. To check for errors occurring at the time of a kernel launch, for example if the launch configuration is erroneous, CUDA provides the `cudaGetLastError` function, which does return a value of type `cudaError_t`.

```cpp
/*
 * This launch should cause an error, but the kernel itself
 * cannot return it.
 */

someKernel<<<1, -1>>>();  // -1 is not a valid number of threads.

cudaError_t err;
err = cudaGetLastError(); // `cudaGetLastError` will return the error from above.
if (err != cudaSuccess)
{
  printf("Error: %s\n", cudaGetErrorString(err));
}
```

Finally, in order to catch errors that occur asynchronously, for example during the execution of an asynchronous kernel, it is essential to check the status returned by a subsequent synchronizing CUDA runtime API call, such as `cudaDeviceSynchronize`, which will return an error if one of the kernels launched previously should fail.

---
### Exercise: Add Error Handling

Currently [`01-add-error-handling.cu`](../edit/06-errors/01-add-error-handling.cu) compiles, runs, and prints that the elements of the array were not successfully doubled. The program does not, however, indicate that there are any errors within it. Refactor the application to handle CUDA errors so that you can learn what is wrong with the program and effectively debug it. You will need to investigate both synchronous errors potentially created when calling CUDA functions, as well as asynchronous errors potentially created while a CUDA kernel is executing. Refer to [the solution](../edit/06-errors/solutions/01-add-error-handling-solution.cu) if you get stuck.

In [4]:
!nvcc -arch=sm_70 -o add-error-handling ../edit/06-errors/01-add-error-handling.cu -run

Error: operation not supported
nvcc error   : './"add-error-handling"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"add-error-handling"' core dumped


---
### CUDA Error Handling Function

It can be helpful to create a macro that wraps CUDA function calls for checking errors. Here is an example, feel free to use it in the remaining exercises:

```cpp
#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

int main()
{

/*
 * The macro can be wrapped around any function returning
 * a value of type `cudaError_t`.
 */

  checkCuda( cudaDeviceSynchronize() )
}
```

---
## Summary

At this point in time you have accomplished all of the following lab objectives:

- Write, compile, and run C/C++ programs that both call CPU functions and **launch** GPU **kernels**.
- Control parallel **thread hierarchy** using **execution configuration**.
- Refactor serial loops to execute their iterations in parallel on a GPU.
- Allocate and free memory available to both CPUs and GPUs.
- Handle errors generated by CUDA code.

Now you will complete the final objective of the lab:

- Accelerate CPU-only applications.

---
### Final Exercise: Accelerate Vector Addition Application

[`01-vector-add.cu`](../edit/07-vector-add/01-vector-add.cu) contains a functioning CPU-only vector addition application. Accelerate its `addVectorsInto` function to run as a CUDA kernel on the GPU and to do its work in parallel. Consider the following that need to occur, and refer to [the solution](../edit/07-vector-add/solutions/01-vector-add-solution.cu) if you get stuck.

In [7]:
!nvcc -arch=sm_70 -o vector-add ../edit/07-vector-add/01-vector-add.cu -run

nvcc error   : './"vector-add"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"vector-add"' core dumped


---
## Advanced Content

---
## Grids and Blocks of 2 and 3 Dimensions

Grids and blocks can be defined to have up to 3 dimensions. Defining them with multiple dimensions does not impact their performance in any way, but can be very helpful when dealing with data that has multiple dimensions, for example, 2d matrices. To define either grids or blocks with two or 3 dimensions, use CUDA's `dim3` type as such:

```cpp
dim3 threads_per_block(16, 16, 1);
dim3 number_of_blocks(16, 16, 1);
someKernel<<<number_of_blocks, threads_per_block>>>();
```

Given the example just above, the variables `gridDim.x`, `gridDim.y`, `blockDim.x`, and `blockDim.y` inside of `someKernel`, would all be equal to `16`.

---
### Exercise: Accelerate 2D Matrix Multiply Application

The file [`01-matrix-multiply-2d.cu`](../edit/08-matrix-multiply/01-matrix-multiply-2d.cu) contains a host function `matrixMulCPU` which is fully functional. Your task is to build out the `matrixMulGPU` CUDA kernel. Refer to [the solution](../edit/08-matrix-multiply/solutions/01-matrix-multiply-2d-solution.cu) if you get stuck.

In [2]:
!nvcc -arch=sm_70 -o matrix-multiply-2d ../edit/08-matrix-multiply/01-matrix-multiply-2d.cu -run

nvcc error   : './"matrix-multiply-2d"' died due to signal 11 (Invalid memory reference)
nvcc error   : './"matrix-multiply-2d"' core dumped


---
### Exercise: Accelerate A Thermal Conductivity Application

In the following exercise, you will be accelerating an application that simulates the thermal conduction of silver in 2 dimensional space.

Convert the `step_kernel_mod` function inside [`01-heat-conduction.cu`](../edit/09-heat/01-heat-conduction.cu) to execute on the GPU, and modify the `main` function to properly allocate data for use on CPU and GPU. The `step_kernel_ref` function executes on the CPU and is used for error checking. Because this code involves floating point calculations, different processors, or even simply reordering operations on the same processor, can result in slightly different results. For this reason the error checking code uses an error threshold, instead of looking for an exact match. Refer to [the solution](../edit/09-heat/solutions/01-heat-conduction-solution.cu) if you get stuck.

In [ ]:
!nvcc -arch=sm_70 -o heat-conduction ../edit/09-heat/01-heat-conduction.cu -run

> Credit for the original Heat Conduction CPU source code in this task is given to the article [An OpenACC Example Code for a C-based heat conduction code](http://docplayer.net/30411068-An-openacc-example-code-for-a-c-based-heat-conduction-code.html) from the University of Houston.